# Imports

In [2]:
from utils import get_block_data, update_notion, find_text, format_text
from config import NOTION_KEY, DEEPL_KEY
from evaluate import load
import deepl

# Load Notion Data

In [3]:
page_url = "https://www.notion.so/Capitolo-1-b965a7473ab845a9a9b826e98a52ca3c"
block_id = page_url.split("-")[-1]
data = get_block_data(NOTION_KEY, block_id)

# Clean Data

In [4]:
# Find index of heading 1 blocks
h1_blocks_idx = [idx for idx, block in enumerate(data['results']) if block['type'] == 'heading_1']

In [5]:
original_text = find_text(data, h1_blocks_idx[0]+1, h1_blocks_idx[1])
my_translation = find_text(data, h1_blocks_idx[1]+1, h1_blocks_idx[2])
given_translation = find_text(data, h1_blocks_idx[2]+1, len(data['results']))

In [6]:
original_text = format_text(original_text)
my_translation = format_text(my_translation)
given_translation = format_text(given_translation)

# Score My Translation

In [7]:
# Compute sacrebleu score
metric = load('sacrebleu')
my_score = metric.compute(
    predictions=[my_translation], 
    references=[given_translation]
)['score']

# Score DeepL Translation

In [8]:
# Translate with DeepL
translator = deepl.Translator(DEEPL_KEY)
deepl_translation = translator.translate_text(
    original_text, 
    target_lang='EN-GB'
)

In [9]:
# Compute DeepL score
deepl_score = metric.compute(
    predictions=[deepl_translation.text], 
    references=[given_translation]
)['score']

# Write Scores To Notion

In [10]:
update_notion(NOTION_KEY, my_score, deepl_score)